In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

kmnist = datasets.KMNIST("data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))

In [2]:
kmnist_loader = DataLoader(kmnist, batch_size=len(kmnist))

In [3]:
kmnist_data = next(iter(kmnist_loader))[0]

In [4]:
kmnist_data.shape

torch.Size([112800, 1, 28, 28])

In [5]:
KMNIST_MEAN = kmnist_data.mean()
KMNIST_STD = kmnist_data.std()
KMNIST_MEAN, KMNIST_STD

(tensor(0.1751), tensor(0.3332))

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_loader = DataLoader(datasets.KMNIST("data", train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((KMNIST_MEAN,), (KMNIST_STD,))
    ])), batch_size=128, shuffle=False)
val_loader = DataLoader(datasets.KMNIST("data", train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((KMNIST_MEAN,), (KMNIST_STD,))
    ])), batch_size=128, shuffle=False)

In [7]:
import pytorch_lightning as pl
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from optimizee.mnist import KMnistConvModel

class KMNISTClassifier(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = KMnistConvModel()
        self.criterion = nn.NLLLoss()
        self.metrics = {
            "accuracy": {
                "train": torchmetrics.Accuracy(),
                "val": torchmetrics.Accuracy()
            }
        }

    def step(self, batch, step_name="train"):
        X, y = batch
        outputs = self.model(X)
        loss = self.criterion(outputs, y)
        preds = self.forward(X)
        metric = self.metrics["accuracy"][step_name]
        metric.update(preds.cpu(), y.cpu())
        metric_val = metric.compute()
        self.log(f"{step_name}_loss", loss, on_epoch=True)
        self.log(f"{step_name}_accuracy", metric_val, on_epoch=True)
        return loss

    def forward(self, X, *args):
        return self.model(X)

    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")
    
    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def predict_step(self, batch, batch_idx):
        X, y = batch
        return self.forward(X)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.hparams.lr)
        return optimizer

In [8]:
import wandb
from pytorch_lightning.loggers import WandbLogger

NUM_EPOCHS = 10

wandb_logger = WandbLogger(project="optml-project", name=f"kmnist")

model = KMNISTClassifier(lr=1e-4)
trainer = pl.Trainer(default_root_dir="models/kmnist", max_epochs=NUM_EPOCHS, logger=wandb_logger, accelerator="gpu")
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mismayil. Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type            | Params
----------------------------------------------
0 | model     | EMnistConvModel | 449 K 
1 | criterion | NLLLoss         | 0     
----------------------------------------------
449 K     Trainable params
0         Non-trainable params
449 K     Total params
1.798     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_accuracy_epoch,▁▅▆▇▇▇████
train_accuracy_step,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
train_loss_epoch,█▃▂▂▂▁▁▁▁▁
train_loss_step,█▅▃▄▃▃▃▃▂▂▂▂▂▂▁▂▁▂▁▂▂▁▂▁▂▁▁▁▁▂▂▁▁▂▁▁▁▁▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▁▄▅▆▇▇▇███
val_loss,█▄▃▂▂▂▁▁▁▁
epoch,9
train_accuracy_epoch,0.83587
train_accuracy_step,0.83843


In [9]:
from collections import OrderedDict
kmnist_model_dict = OrderedDict({name.replace("model.", ""): parameter for name, parameter in model.state_dict().items()})
torch.save(kmnist_model_dict, "ckpt/attack_model/kmnist_cnn.pt")

In [17]:
preds = trainer.predict(model, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/.conda/envs/optml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 882it [00:00, ?it/s]

In [18]:
preds = torch.cat(preds)

In [19]:
preds.shape

torch.Size([18800, 47])

In [20]:
preds = preds.argmax(dim=1)

In [21]:
val_targets = []

for _, y in val_loader:
    val_targets.append(y)

val_targets = torch.cat(val_targets)

In [22]:
val_targets.shape

torch.Size([18800])

In [23]:
(preds == val_targets).sum() / len(val_targets)

tensor(0.8671)

In [24]:
correct_indices = torch.where(preds == val_targets)[0]

In [25]:
import numpy as np

with open("data/kmnist_correct/label_correct_index.npy", "wb") as f:
    np.save(f, correct_indices.numpy())